In [1]:
!git clone https://github.com/jerryjliu/llama_index.git

Clonage dans 'llama_index'...
remote: Enumerating objects: 34332, done.
remote: Counting objects: 100% (33988/33988), done.
remote: Compressing objects: 100% (9965/9965), done.
remote: Total 34332 (delta 24102), reused 32311 (delta 23340), pack-reused 344
Réception d'objets: 100% (34332/34332), 69.34 Mio | 18.59 Mio/s, fait.
Résolution des deltas: 100% (24123/24123), fait.
Mise à jour des fichiers: 100% (1491/1491), fait.


In [2]:
!pip install llama-index llama_hub wikipedia

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [29]:
import openai
import os

OPENAI_API_KEY = 

openai.api_key = OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [3]:
from pathlib import Path
from llama_index import download_loader

ImageReader = download_loader("ImageReader")

# If the Image has key-value pairs text, use text_type = "key_value"
loader = ImageReader(text_type = "key_value")
documents = loader.load_data(file=Path('./Praxedo/000000097250343-P-3916532241.jpg'))

# # If the Image has plain text, use text_type = "plain_text"
# loader = ImageReader(text_type = "plain_text")
# documents = loader.load_data(file=Path('./image.png'))

  Using cached sentencepiece-0.1.99-cp39-cp39-macosx_10_9_x86_64.whl (1.2 MB)
  Using cached pytesseract-0.3.10-py3-none-any.whl (14 kB)


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


RuntimeError: Failed to import transformers.models.donut.image_processing_donut because of the following error (look up to see its traceback):
Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [4]:
from llama_hub.wikipedia.base import WikipediaReader

loader = WikipediaReader()
documents = loader.load_data(pages=['Berlin', 'Rome', 'Tokyo', 'Canberra', 'Santiago'])

In [7]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('../../Downloads/untitled folder 2').load_data()
index = VectorStoreIndex.from_documents(documents)

In [8]:
documents

[Document(id_='82876abf-b624-4cc4-b5dd-c754c749e53a', embedding=None, metadata={'page_label': '1', 'file_name': 'All_documents.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='92a361a4bb9810db273f09c050a3cfff988f680bbd31ab6baa71c2b4d5a8f938', text="IDENTITY\nPHOTOGRAPH\n20. If  you are resident in a country other than the country of  current nationality, please state \xa0:\n       Number of  residence permit                   Date of  issue                                         Valid until               \n21. Current occupation    \n23. I request a visa for the following purpose \xa0:   \uf063  Employment  \uf063  Studies  \uf063  Training period/education\uf063  Marriage  \uf063  Medical reasons      \uf063  Family stay  \uf063  Private stay/Visitor  \uf063  Re-entry visa      \uf063  Official taking up of  duties  \uf063  Other (please specify) :  .........…..............…....…..…..........….….........… .......OFFICIAL DECISION\nDate\xa

In [23]:
query_engine = index.as_query_engine()
response = query_engine.query("Give more info about Charlene")
print(response)

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

In [39]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
# Using LlamaIndex as a Callable Tool
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

from llama_index import VectorStoreIndex, SimpleDirectoryReader
documents = SimpleDirectoryReader('llama_index/examples/paul_graham_essay/data').load_data()
index = VectorStoreIndex.from_documents(documents=documents)
tools = [
    Tool(
        name = "LlamaIndex",
        func=lambda q: str(index.as_query_engine().query(q)),
        description="useful for when you want to answer questions about the author. The input to this tool should be a complete english sentence.",
        return_direct=True
    ),
]

# set Logging to DEBUG for more detailed outputs
memory = ConversationBufferMemory(memory_key="chat_history")
llm = ChatOpenAI(temperature=0)
agent_executor = initialize_agent(tools, llm, agent="conversational-react-description", memory=memory)
agent_executor.run(input="hi, i am bob")

'Hello Bob! How can I assist you today?'

In [40]:
agent_executor.run(input="What did the author do growing up?")

"The author's childhood involved working on writing and programming outside of school. They wrote short stories and began programming on an IBM 1401 in 9th grade. They used an early version of Fortran and had to type programs on punch cards. They also expressed their fascination with microcomputers and eventually convinced their father to buy a TRS-80. Additionally, they mentioned their initial plan to study philosophy in college but later switched to AI."

In [48]:
"""Example of using a QnA chain with guardrails."""
import logging
import os

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

from nemoguardrails import LLMRails, RailsConfig

# logging.basicConfig(level=logging.INFO)

# COLANG_CONFIG = """
# define user express greeting
#   "hi"

# define user express insult
#   "You are stupid"

# # Basic guardrail against insults.
# define flow
#   user express insult
#   bot express calmly willingness to help

# # Here we use the QA chain for anything else.
# define flow
#   user ...
#   $answer = execute qa_chain(query=$last_user_message)
#   bot $answer

# """

# YAML_CONFIG = """
# models:
#   - type: main
#     engine: openai
#     model: text-davinci-003
# """


# def _get_qa_chain(llm):
#     """Initializes a QA chain using the jobs report.

#     It uses OpenAI embeddings.
#     """
#     loader = TextLoader(
#         os.path.join(
#             os.path.dirname(__file__),
#             "..",
#             "examples/grounding_rail/kb/report.md",
#         )
#     )
#     docs = loader.load()
#     text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
#     texts = text_splitter.split_documents(docs)

#     embeddings = OpenAIEmbeddings()
#     docsearch = Chroma.from_documents(texts, embeddings)

#     qa_chain = RetrievalQA.from_chain_type(
#         llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
#     )

#     return qa_chain

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/macbookpro/opt/anaconda3/envs/python39/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/9z/vxzd5t7x2vn4503l8w3td5pm0000gn/T/ipykernel_28298/1255526273.py", line 11, in <module>
    from nemoguardrails import LLMRails, RailsConfig
  File "/Users/macbookpro/opt/anaconda3/envs/python39/lib/python3.9/site-packages/nemoguardrails/__init__.py", line 18, in <module>
    from .rails import LLMRails, RailsConfig
  File "/Users/macbookpro/opt/anaconda3/envs/python39/lib/python3.9/site-packages/nemoguardrails/rails/__init__.py", line 17, in <module>
    from .llm.llmrails import LLMRails
  File "/Users/macbookpro/opt/anaconda3/envs/python39/lib/python3.9/site-packages/nemoguardrails/rails/llm/llmrails.py", line 38, in <module>
    from nemoguardrails.llm.providers import get_llm_provider, get_llm_provider_names
  File "/Users/macbookpro/

In [43]:
"""Demo of using a chain as a custom action."""
config = RailsConfig.from_content(COLANG_CONFIG, YAML_CONFIG)
app = LLMRails(config)

NameError: name 'RailsConfig' is not defined

In [ ]:


# Create and register the chain directly as an action.
qa_chain = _get_qa_chain(app.llm)
app.register_action(qa_chain, name="qa_chain")

# Change to mode here to experiment with the multiple ways of using the chain

# mode = "chain"
mode = "chain_with_guardrails"
# mode = "chat_with_guardrails"

if mode == "chain":
    query = "What is the current unemployment rate?"
    result = qa_chain.run(query)

    print(result)

elif mode == "chain_with_guardrails":
    history = [
        {"role": "user", "content": "What is the current unemployment rate?"}
    ]
    result = app.generate(messages=history)
    print(result)

elif mode == "chat_with_guardrails":
    history = []
    while True:
        user_message = input("> ")

        history.append({"role": "user", "content": user_message})
        bot_message = app.generate(messages=history)
        history.append(bot_message)

        # We print bot messages in green.
        print(f"\033[92m{bot_message['content']}\033[0m")


if __name__ == "__main__":
demo()